In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
import numpy as np
from TransNetV2.inference import TransNetV2

2023-08-06 17:39:20.148637: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-08-06 17:39:20.180679: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 17:39:20.654642: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
!git clone https://github.com/soCzech/TransNetV2.git
!pip install ffmpeg-python pillow

Cloning into 'TransNetV2'...
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 362 (delta 4), reused 4 (delta 4), pack-reused 355
Receiving objects: 100% (362/362), 98.15 KiB | 6.54 MiB/s, done.
Resolving deltas: 100% (210/210), done.
Filtering content: 100% (3/3), 34.77 MiB | 26.10 MiB/s, done.


In [29]:
dir = f"/media/madziegielewska/Seagate Expansion Drive/MAGISTERKA/diploma-project/"

model = TransNetV2()

[TransNetV2] Using weights from /media/madziegielewska/Seagate Expansion Drive/MAGISTERKA/diploma-project/TransNetV2/inference/transnetv2-weights/.


In [31]:
# read all videos

dir_path = 'videos'
files = []

for path in os.listdir(dir_path):
    if os.path.isfile(os.path.join(dir_path, path)):
        files.append(path)

files.sort()
len(files)

899

In [34]:
event_change_test = pd.DataFrame(columns=["video", "frame"])

results = []
count = 0

for file in files:
    print(f"{len(files)-count} left")
    file_path = f"{dir}videos/{file}"

    filename, file_extension = os.path.splitext(f'{file}')
    output_path_preds = f"{dir}Event-Boundary-Detection/transnet_results/predictions/{filename}"
    output_path_sce = f"{dir}Event-Boundary-Detection/transnet_results/scenes/{filename}"

    video_frames, single_frame_predictions, all_frame_predictions = model.predict_video(file_path)

    scenes = model.predictions_to_scenes(single_frame_predictions, threshold=0.005)
    np.savetxt(output_path_sce + "_scenes.txt", scenes, fmt="%d")

    predictions = np.stack([single_frame_predictions, all_frame_predictions], 1)
    np.savetxt(output_path_preds + ".predictions.txt", predictions, fmt="%.6f")

    event_change_pred = np.argmax(single_frame_predictions)

    df = pd.DataFrame([{"video": f'{file}', "frame": f'{event_change_pred}'}])
    event_change_test = pd.concat([event_change_test, df])

    count += 1

# save test preprocessed data to csv
event_change_test.to_csv('test_data_transnet.csv', index=False)

899 left
[TransNetV2] Extracting frames from /media/madziegielewska/Seagate Expansion Drive/MAGISTERKA/diploma-project/videos/100_Trim1.mp4
[TransNetV2] Processing video frames 1042/1042
898 left
[TransNetV2] Extracting frames from /media/madziegielewska/Seagate Expansion Drive/MAGISTERKA/diploma-project/videos/100_Trim2.mp4
[TransNetV2] Processing video frames 1129/1129
897 left
[TransNetV2] Extracting frames from /media/madziegielewska/Seagate Expansion Drive/MAGISTERKA/diploma-project/videos/100_Trim3.mp4
[TransNetV2] Processing video frames 1373/1373
896 left
[TransNetV2] Extracting frames from /media/madziegielewska/Seagate Expansion Drive/MAGISTERKA/diploma-project/videos/100_Trim4.mp4
[TransNetV2] Processing video frames 919/919
895 left
[TransNetV2] Extracting frames from /media/madziegielewska/Seagate Expansion Drive/MAGISTERKA/diploma-project/videos/101_Trim1.mp4
[TransNetV2] Processing video frames 1110/1110
894 left
[TransNetV2] Extracting frames from /media/madziegielewska

In [2]:
# read true data

df = pd.read_csv('Event-Boundary-Detection/data.csv')

videos = df['video'].tolist()
event_change_true = df['frame'].tolist()

In [3]:
# read test data

df = pd.read_csv('test_data_transnet.csv')

videos = df['video'].tolist()
event_change_test = df['frame'].tolist()

In [5]:
# compare results

correct_detection = 0

for true_change, test_change in zip(event_change_true, event_change_test):
    if (true_change == test_change):
        correct_detection += 1
        #print("correct")
    else:
        # include measurement uncertainty
        lower_tolerance = 10
        upper_tolerance = 10

        if(true_change-lower_tolerance < test_change < true_change+upper_tolerance):
            correct_detection += 1


print("correct detection: ", correct_detection)
print("incorrect detection: ", len(event_change_true) - correct_detection)
print(f"accuracy: {round(correct_detection/len(event_change_true)*100, 2)}%")

correct detection:  777
incorrect detection:  120
accuracy: 86.62%
